<center><img src="https://yt3.ggpht.com/ytc/AKedOLRcwXL5heetbKNzpLCY3LOgml-72EcmpALhcvry5g=s900-c-k-c0x00ffffff-no-rj" alt="Logo" width="80" height="80"></center>

<center><h3> Part IV: Modeling </h3></center>

#### Background

In [ ]:
# Copy/Paste some markdown from other file.

.

#### imports

In [ ]:
# use flake to clean up imports

In [255]:
import spacy
import numpy as np
import spacy_transformers
import spacy_universal_sentence_encoder
import en_core_web_sm

pd.options.display.max_colwidth = None
pd.set_option('display.max_columns', None)

#nlp = spacy.load('en_use_md')
nlp = en_core_web_sm.load()

.

#### More preprocessing

In [256]:
usa = pd.read_csv('./data/clean/c_tiktok_US.csv')
ca = pd.read_csv('./data/clean/c_tiktok_CA.csv')
ie = pd.read_csv('./data/clean/c_tiktok_IE.csv')
au = pd.read_csv('./data/clean/c_tiktok_AU.csv')
jm = pd.read_csv('./data/clean/c_tiktok_JM.csv')

In [257]:
df = pd.concat([usa, ca, ie, au, jm], axis=0)

.

Fill NaNs for text columns with "" and remove all numbers from strings.

In [258]:
df = df[['desc', 'aweme_id', 'hashtags']].fillna('')
df.head(4)

,desc,aweme_id,hashtags
0,,7101483721580105006,
1,Ouuuuuu I love coming back to the boot with stink🥰😍😍😍 #louisiana #fyp #foryou,7102443016685833518,#louisiana #fyp #foryou
2,Play the link in the bio to start a Weightloss Journey off the right way for the last time,7097180330989341995,
3,"“Nadine, honey is that you?” Kicking off the weekend with this 1982 performance of Nadine at The Roxy! #Nadine #ChuckBerry #TheRoxy #musictok #livemusic #guitartok",7102429766724128046,#Nadine #ChuckBerry #TheRoxy #musictok #livemusic #guitartok


In [259]:
df['desc']=df['desc'].str.replace(r'[0-9]+', '', regex=True).astype('str')

.

Preserving hashtags when getting word lemma since they're important to keep the way they are. 

In [260]:
# import re
# # Retrieve the default token-matching regex pattern
# re_token_match = spacy.tokenizer._get_regex_pattern(nlp.Defaults.token_match)
# # Add #hashtag pattern
# re_token_match = f"({re_token_match}|#\\w+)"
# nlp.tokenizer.token_match = re.compile(re_token_match).match
# source: link to old source

default_prefixes = list(nlp.Defaults.prefixes)
default_prefixes.remove('#')

prefix_regex = spacy.util.compile_prefix_regex(default_prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

df['desc_lemma'] = df['desc'].apply( lambda x: [y.lemma_ for y in nlp(x)] ).str.join(" ")

In [261]:
df.head(4)

,desc,aweme_id,hashtags,desc_lemma
0,,7101483721580105006,,
1,Ouuuuuu I love coming back to the boot with stink🥰😍😍😍 #louisiana #fyp #foryou,7102443016685833518,#louisiana #fyp #foryou,Ouuuuuu I love come back to the boot with stink 🥰 😍 😍 😍 #louisiana #fyp #foryou
2,Play the link in the bio to start a Weightloss Journey off the right way for the last time,7097180330989341995,,play the link in the bio to start a Weightloss Journey off the right way for the last time
3,"“Nadine, honey is that you?” Kicking off the weekend with this performance of Nadine at The Roxy! #Nadine #ChuckBerry #TheRoxy #musictok #livemusic #guitartok",7102429766724128046,#Nadine #ChuckBerry #TheRoxy #musictok #livemusic #guitartok,""" Nadine , honey be that you ? "" kick off the weekend with this performance of Nadine at the Roxy ! #nadine #ChuckBerry #theroxy #musictok #livemusic #guitartok"


In [262]:

# spacy - other functions for context?

In [263]:
df.set_index('aweme_id', inplace=True)

.

Added custom stopwords for common gen z slang and some words that aren't very helpful or informative. these words are so puplar they don't really say anything distinctive about the people who use them

I kept the jamaican patois ('fi', 'mi', 'ar', dey', etc) in the data because it might return higher similarity scores for other vids that use jamaican slang. except I did remove words that are also used commonly in usa ('yah' for 'yeah')

Also keeping the #trend hashtag

In [264]:
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

extra_words = ['fy', 'fyp', 'foryoupage', 'fypage', 'foryou', 'fypシ', 'viral', 'trending',
                                                              'trendingvideo', 'viralvideo', 'tiktok', 'like', 'don',
                            'lol', 'follow', 'time', 'day', 'replay', 'video', 'stop', 'want', 'got', 'best', 'love',
                            'stitch', 'reply', 'wait', 'come', 'bio', 'let', 'right', 'left', 'guy', 'guys', 'fun', 'things', 'end',
                            'going', 'year', 'know', 'good', 'new', 'll', 'need', 'happy', 'way', 'tho', 'help', 'today', 'days',
                            'live', 'feel', 'me', 'link', 'tho', 'nah', 'fyppppppppppppppppppppppp', 'fypp', 'find',  'gone',
                            'gotta', 'gonna', 'goviral', 'aint', 'yall', 'hi', 'heart', 'hear', 'hours', 'join', 'late', 'post', 'pls',
                            'st', 'tok', 'oh', 'ok', 'okay', 'said', 'sad', 'rip', 'yes', 'no', 'ya', 'yeah', 'yea', 'weeks', 'viraltiktok',
                            'tell', 'thank', 'ur', 'your', 'im', 'dis', 'idk', 'og', 'ib', 'ig', 'ob', 'iykyk', 'lmao', 'lmfao',
                            'link', 'likes', 'quick', 'seen', 'sure', 'tag', 've', 'm', 'plz', 'soon', 'isn', 'went', 'go', 'vs',
                            'have', 'having', 'happen', 'happened', 'go', 'goes', 'com', 'bit', 'bc', 'ad', 'ai', 'far', 'fave',
                             'fav', 'friend', 'great', 'guess', 'hard', 'held', 'hit', 'hold', 'leg', 'later', 'leave', 'left', 'life', 
                             'lot', 'little', 'look', 'long', 'lil', 'meet', 'miss', 'nice', 'nd', 'old', 'page', 'point', 'possible', 'pt',
                             'ready', 'real', 'set', 'st', 'start', 'stop', 'small', 'big', 'thing', 'think', 'tik', 'till', 'tip', 'talk',
                             'true', 'tt', 'try', 'turn', 'right', 'wrong', 'ago', 'bestie', 'bro', 'damn', 'vid', 'man', 'foryourpage',
                               'edit', 'xyzbca', 'zyxcba', 'zyzz', 'a', 'ab', 'abc', 'check', 'podcast', 'podcasts', 'share', 'walk', 'walks',
                               'walked'
                            ]

extra_hash = ['#'+i for i in extra_words]


print(len(stopwords))
stopwords = stopwords.union(extra_words).union(extra_hash)



326


.


Countvectorized text with token pattern that keeps hashtags.


In [265]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

cv = CountVectorizer(strip_accents=False, 
                     min_df = 3,
                     max_df = 0.5,
                     stop_words=stopwords,
                     token_pattern=r'\b\w\w+\b|(?<!\w)@\w+|(?<!\w)#\w+'
                    )

X = df['desc_lemma'].values.astype('U')
X_cv = pd.DataFrame(cv.fit_transform(X).A,  columns=[x for x in cv.get_feature_names_out()], index=df.index)

df_cv = df.copy()#.set_index('aweme_id')

for column in X_cv.columns:
    df_cv[column]=X_cv[column]
    
# maybe use concat instead because of error PerformanceWarning: DataFrame is highly fragmented.
# Was still have issues with countvectorized cols joining incorrectly.

/var/folders/zd/6jtvs5r55vvdpnl2c2q7kv0m0000gn/T/ipykernel_90554/281019293.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cv[column]=X_cv[column]


.


Amazingly, the Jamaican patois survived being converted to american lemma. Might need to check for longer slang words to make sure they weren't lemmatized wrong.

Scroll right to see features.

In [266]:
df_cv.head(30)

desc  \
aweme_id                                                                                                                                                                                                                                                                       
7101483721580105006                                                                                                                                                                                                                                                            
7102443016685833518                                                                                                                                                                            Ouuuuuu I love coming back to the boot with stink🥰😍😍😍 #louisiana #fyp #foryou   
7097180330989341995                                                                                                                                                               Play the link in the bio to start a Weightloss Journey off the right way for the last time   
7102429766724128046                                                                                          “Nadine, honey is that you?” Kicking off the weekend with this  performance of Nadine at The Roxy! #Nadine #ChuckBerry #TheRoxy #musictok #livemusic #guitartok   
7102551198959422762                                                                          I can hear my daughter now..."Dad! STOP RAPPING! You're old!l 😂😂😩 @logic #trendingsong #kids #parenting #fyp #pop #newmusic #memorialdayweekend #memorialday #dance #dancemusic   
7102442784753241386                                                                                                       Were we able to join you for an evening this tour? Thank you to all who made us feel loved these past  weeks! Let us know where we should go next!   
7102468759616539950                                                                                                                                                                                                              it’s about that time again…. 🎡#countrymusic   
7102515825147530542                                                                                                                      The seeds have been planted @michaelbrun MichaelBrun is the future 🙏🏾 🇭🇹 #apmworldmag 📷Stay tune …….. #haiti #bayotour #fyp #foryou   
7101017494013414698                                                                                                                                                                                      Girls don’t always come back 🪃 #boomerang #girlsbelike #guys #trend   
7102297231226866990                                                                                                                                                                                         Drafty #fy #fyp #jumpintheline @officialhelene #hairspraymusical   
7103552205994904878                                                                                                                                                                                                                              The Power of Being Yourself   
7102328890508725550                                                                                                                                                                                                                                                        🫣   
7103597955126693166                                                                                                                                                                 COMMENT YOUR FAVORITE DONUT. #atreyu #music #band #rockmusic #tour #donut #food #foodtok   
7102449300973587754                                                                                                                                                                     

In [267]:
#hashs= [i for i in df.columns if i.startswith('hash_')]

In [268]:
top10 = pd.DataFrame(df_cv.drop(columns=['desc', 'desc_lemma', 'hashtags']).sum()).sort_values(by=0, ascending=False).head(20).reset_index().rename(
        columns={'index': "word", 0: "count"});
top10['word']=top10['word']#.str.replace("hash_", "#");
top10

,word,count
0,#funny,100
1,#trende,81
2,#johnnydepp,55
3,#comedy,49
4,#jamaicantiktok,45
5,#duet,43
6,#jamaicatiktok,41
7,#meme,40
8,#amberheard,37
9,#justiceforjohnnydepp,34


.

In [269]:
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity



cs= cosine_similarity(X=df_cv.drop(columns=['hashtags', 'desc', 'desc_lemma']), Y=df_cv.drop(columns=['hashtags', 'desc', 'desc_lemma']))


rec = pd.DataFrame(cs, columns=df.index, index=df.index)

rec.head(10)

aweme_id             7101483721580105006  7102443016685833518  \
aweme_id                                                        
7101483721580105006                  0.0                  0.0   
7102443016685833518                  0.0                  0.0   
7097180330989341995                  0.0                  0.0   
7102429766724128046                  0.0                  0.0   
7102551198959422762                  0.0                  0.0   
7102442784753241386                  0.0                  0.0   
7102468759616539950                  0.0                  0.0   
7102515825147530542                  0.0                  0.0   
7101017494013414698                  0.0                  0.0   
7102297231226866990                  0.0                  0.0   

aweme_id             7097180330989341995  7102429766724128046  \
aweme_id                                                        
7101483721580105006                  0.0                  0.0   
7102443016685833518                  0.0                  0.0   
7097180330989341995                  1.0                  0.0   
7102429766724128046                  0.0                  1.0   
7102551198959422762                  0.0                  0.0   
7102442784753241386                  0.0                  0.0   
7102468759616539950                  0.0                  0.0   
7102515825147530542                  0.0                  0.0   
7101017494013414698                  0.0                  0.0   
7102297231226866990                  0.0                  0.0   

aweme_id             7102551198959422762  7102442784753241386  \
aweme_id                                                        
7101483721580105006                  0.0                  0.0   
7102443016685833518                  0.0                  0.0   
7097180330989341995                  0.0                  0.0   
7102429766724128046                  0.0                  0.0   
7102551198959422762                  1.0                  0.0   
7102442784753241386                  0.0                  1.0   
7102468759616539950                  0.0                  0.0   
7102515825147530542                  0.0                  0.0   
7101017494013414698                  0.0                  0.0   
7102297231226866990                  0.0                  0.0   

aweme_id             7102468759616539950  7102515825147530542  \
aweme_id                                                        
7101483721580105006                  0.0                  0.0   
7102443016685833518                  0.0                  0.0   
7097180330989341995                  0.0                  0.0   
7102429766724128046                  0.0                  0.0   
7102551198959422762                  0.0                  0.0   
7102442784753241386                  0.0                  0.0   
7102468759616539950                  1.0                  0.0   
7102515825147530542                  0.0                  1.0   
7101017494013414698                  0.0                  0.0   
7102297231226866990                  0.0                  0.0   

aweme_id             7101017494013414698  7102297231226866990  \
aweme_id                                                        
7101483721580105006                  0.0                  0.0   
7102443016685833518                  0.0                  0.0   
7097180330989341995                  0.0                  0.0   
7102429766724128046                  0.0                  0.0   
7102551198959422762                  0.0                  0.0   
7102442784753241386                  0.0                  0.0   
7102468759616539950                  0.0                  0.0   
7102515825147530542                  0.0                  0.0   
7101017494013414698                  1.0                  0.0   
7102297231226866990                  0.0                  1.0   

aweme_id             7103552205994904878  7102328890508725550  \
aweme_id                      

.

"""is there a way to add another layer, so it knows #irishtiktok and #irishcomedy are related? OR to update row values based on how often words appear together in the text?

Prioritize hashtag matchnig over the rest of the text?

Couldn't find a model that did well with compund words, and had trouble using nlp to get cosine sim of whole dataframe."""

.

In [270]:
rec[(rec[7097180330989341995]>0) & (rec[7097180330989341995]<1)][7097180330989341995] 

aweme_id
7097180330989341995    1.000000
7102988658902420779    0.235702
7102162709243776299    0.707107
7102152412756020526    0.500000
7101538756171287851    0.577350
7101815420872674602    0.408248
7101826660877733166    0.162221
7096259626940763398    0.288675
7094351899314457899    0.235702
7089927728060714246    0.707107
7095843346228202798    0.235702
7102441357138038022    0.408248
7072404476388330794    0.408248
7101590725422337282    0.408248
7100212076576115973    0.500000
Name: 7097180330989341995, dtype: float64

.

In [250]:
def function(post_id):
    filter = rec[(rec[post_id]>0) & (rec[post_id]<=1)][post_id].sort_values(ascending=False).reset_index()
    df3 = pd.DataFrame(filter).rename(columns={'aweme_id':'video id', post_id:'similarity'}).head(10)
    df4 = pd.merge(left=df3, right=df_cv['desc'], left_index=False, left_on='video id', right_index=True)
    return(df4)
    #left join video URL, music info, fyp_location, region
    #rename this function
    #need pandas formatting library to update first row

.

Recommender is very good at recommending more Jamaican videos if it picks up on a little patois in the description.

In [251]:
function(7094735704051305733)

,video id,similarity,desc
0,7094735704051305733,1.000000,Him naah leff mi out💯🙌🏾❤️#perryinchrist #stayinchrist #jamaica #christian #viral #caribbean #jamaicatiktok
1,7098054758610652421,0.547723,#jamaica #foryourpage #foryou #fyp #jamaicatiktok #jamaicatiktok #thevoice #tessannechin
2,7093585398084947206,0.516398,#yardiechallenge @Jazzy Vybz Muzik #djjazzy #jazzyvybzmuzik #dancehall 🇯🇲 @n_asti.a #riflewalk #leggodibird #weddi #jiggy #jamaicatiktok #fyp #jamaica #viral #caribeantiktok #dancehallvibes
3,7092501240898424110,0.516398,When Ella won’t listen to me #jamaica #jamaicatiktok #fyp
4,7072816142208978222,0.516398,Beef Patties. #jamaica #jamaicatiktok #proudjamaican #jamaicancooking #jamaicanfood #cookingtiktok
5,7101806922424225029,0.516398,Biggsdon New Chune Preview 🔥🔥 #fy #fyp #foryoupage #fypシ #dancehall #reggae #biggsdon #biggsdonchallenge #jamaica #jamaicatiktok
6,7098095190094777606,0.507093,Dropsy run inna mi family bbz😔 #fyp #foryoupage #viral #fypシ #trend #text #search #phone #relationship #caribbean #jamaica #swizzledioptimist
7,7091472593290562821,0.447214,Caption dis😂#fyp #viral #unknown__iix #derrickmorgan #foryou #jamaicatiktok
8,7081050152735165701,0.447214,Co mi Likkle rass #fyp #foryoupage🇯🇲🇧🇧 #tiiaababy #bigbattytiiaa
9,7090963964716125446,0.447214,Lol mi cyaa badda 🤣


.

Somehow this recommender knew that #musiciansoftiktok and #artistsoftiktok were similar. It also grabbed the #singersongwriter tag. That is desired behavior but I do not know how it's doing it. I don't think I coded this in, and hashtags weren't taken down to the lemma. 

In [223]:
function(7102894898080304430)

,video id,similarity,desc
0,7102146577741073710,0.333333,Practice makes progress folks! #fyp #bridgerton #musiciansoftiktok #cello
1,7100050747319995650,0.333333,#fyp #viral #busker #melbourne #inoxia #americanboy #OscarsAtHome #artistsoftiktok
2,7101832423817628970,0.288675,Take me back #traveltiktok #travel #spain #foryou #fypシ #artistsoftiktok #artistatiktok #boricua
3,7103681011262754054,0.288675,coming out June rd 🖤👯‍♀️ #countrymusic #coverart #singersongwriter #newmusic #sayittomyself
4,7102166885067754795,0.258199,We got to celebrate #EverythingButOn going #… it was a great time being around everyone who works hard to keep this ship goin. Thank y’all for streaming 🤘🏻 #countrymusic #countrysong #singersongwriter


.

In [254]:
function(7101754758817516806)

,video id,similarity,desc
0,7101754758817516806,1.000000,Any girls ever have this issue at the gym?! 😂 @heyitzhailzz_ #gymtok #gymlife #fittok
1,7091345141008502021,0.670820,This trend was made for gym girls #fyp #gymtok
2,7096300319100898566,0.632456,just positive gym things😊. #FitTok #gymculture #workouttips #gymlife
3,7102934095961001259,0.516398,Filming in the gym does NOT give you priority over others. #MYOB #gymtok
4,7103235615285693738,0.516398,Info in comments #proteinpancakes #pancakes #protein #bulking #FitTok #gymtok
5,7098811142595693825,0.447214,I got the girls with big girls 🤍
6,7097359634490510597,0.447214,That one girl on our FYP😭😂🔥
7,7099535560254967046,0.447214,#fyp #foryou #kylecookefitness #viral #gymtok
8,7093953318384307461,0.447214,Best girl #foryou #fyp #besties @a.m.y.l.e.i.g.h
9,7094569957421223174,0.447214,Tik tok do your thing! Let’s help a girl out #norway #cheater #girlssupportgirls


.

Note that if a video has NO features that passed the preprocessing steps, it will not even return as similar to itself. These rows/columns could be removed from the dataframe eventually.

In [224]:
function(7102443016685833518)

,video id,similarity,desc
aweme_id,,,


.

Next steps:
    
    * Adding post author, FYP location, origin country of video, and song title/artist to recommender. 
    
    * Building a model that returns relevant videos based on user entry (textbox) or dropdown selection of their interests.
    
    * Web app that allows user to enter interests, and plays relevant videos.
    
    * Do an API pull for videos; see how closely tiktok's list of recommended (similar) videos matches mine.
    
    

.

    
    
Ideas to improve performance:
   
    * I would like to find a model that was trained on social media (twitter or instagram). Despite returning videos with relevant descriptions, this recommender misses some connections due to issues understanding and comparing compound words.
   
    * (a test of models that were built for cosine similarity showed that they did not include support for parsing compound words - i.e. most compound hashtags "#irishhumor" had a low similarity score when comparing to hashtag "irish".)
    
    * Adding another layer (?) of meaning by using nlp on description text, then comparing word vectors to build recommender. This might help with homonym issues and returning videos by similar overarching categories. I do not know if this is possible. 
    

.

Takeaways:
    
    * Jamaica is a good example of where nlp alone could be used to determine post relevancy to a user rather than strict location barriers - users might be interested in contect from "less popular" creators (not shown on international FYP pages), if the creator's content matches their interests.
    
    * For Jamaica this might be somewhat localized, but for other examples there are cultural characteristics that can be inferred from text features to determine relevancy to a user. How this would contribute to users getting stuck in certain "bubbles" or "echo chambers" of content is a separate ethics discussion.